# Generate example static catalog for S1 RTC on AWS

https://registry.opendata.aws/sentinel-1-rtc-indigo/

| Location | MGRS |
| - | - | 
| West coast (WA state) | 10UCU |
| US-canada border | 10UFV |
| Central UTM boundary 12/13 (Colorado) | 12SYJ, 13SBD | 
| Gulf coast | 17RMJ, 15RUN |
| East coast | 19TCH |

With the following nested catalog structure:
```
.
├── <MGRS>
│   └── <YEAR>
│       ├── S1A_20170102_19TCH_ASC.json
│       ├── S1A_20170107_19TCH_ASC.json
│       └── S1A_20170114_19TCH_ASC.json
└── catalog.json
```

In [ ]:
# Requires https://github.com/scottyhq/stactools/tree/sentinel1
#!pip install --no-deps -e ../stactools

In [ ]:
import s3fs

import pystac
from pystac import Catalog, Collection, CatalogType
from pystac.layout import TemplateLayoutStrategy
from stactools.sentinel1.stac import create_item, create_collection


%load_ext autoreload
%autoreload 2

In [ ]:
s3 = s3fs.S3FileSystem(anon=True)

In [ ]:
# Load previously saved paths 
with open('paths.txt') as f:
    paths = [x.rstrip() for x in f.readlines()]

In [ ]:
paths[:4]

In [ ]:
def s3_to_http(s3path, region='us-west-2'):
    s3prefix = 'sentinel-s1-rtc-indigo'
    newprefix = f'https://sentinel-s1-rtc-indigo.s3.{region}.amazonaws.com'
    http = s3path.replace(s3prefix, newprefix)
    #print(http)
    return http

In [ ]:
# Test creating single item
#print(paths[0])
#s3_to_http(f'{paths[0]}')

#import rasterio
#with rasterio.open(s3_to_http(f'{paths[0]}')+'/local_incident_angle.tif') as src:
#    print(src.profile)

In [ ]:
%%time
item = create_item(s3_to_http(f'{paths[0]}'))

In [ ]:
# View the JSON
#import json
#print(json.dumps(item.to_dict(), indent=1))

# Validate in-memory STAC
#item.validate()

# Validate on-disk STAC
# pystac.read_file(f'{dst}/{dst}.json').validate()

In [ ]:
%%time 
# Generate all the items serially (could easily parallelize)
items = [create_item(s3_to_http(f'{i}')) for i in paths]

In [ ]:
# Add items to a catalog
#catalog = Catalog(id='sentinel1-rtc',
#                  description='Sentinel1 radiometric terrain corrected backscatter (RTC)'
#                 )

#catalog.add_items(items)
#print(len(catalog)) #TypeError: object of type 'Catalog' has no len()
#catalog.describe()

# Save the catalog in current directory
#catalog.normalize_hrefs('./')
#catalog.validate()
#catalog.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED)

In [ ]:
collection = create_collection()
collection.add_items(items)
collection.describe()

In [ ]:
# Save the catalog in current directory
template = '${sentinel:mgrs}/${year}'
collection.generate_subcatalogs(template=template)
collection.describe()

In [ ]:
collection.normalize_hrefs('./')
collection.validate()

In [ ]:
collection.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED)

# Modify existing catalog

In [ ]:
#catalog = pystac.read_file('catalog.json')

In [ ]:
#item = catalog.get_item('S1A_20170101_10UCU_ASC')
#item.properties

In [ ]:
#template = '${sentinel:utm_zone}${sentinel:latitude_band}${sentinel:grid_square}/${year}'
#strategy = TemplateLayoutStrategy(item_template=template)
#catalog.normalize_hrefs('./test', strategy=strategy)
#catalog.save(catalog_type=CatalogType.SELF_CONTAINED)